In [ ]:
import pickle
from pathlib import Path

import cv2
import isort
import matplotlib.pyplot as plt
import numpy as np
import torch
from pytorch3d.renderer import (AlphaCompositor, FoVOrthographicCameras,
                                NormWeightedCompositor,
                                PointsRasterizationSettings, PointsRasterizer,
                                PointsRenderer, PulsarPointsRenderer,
                                look_at_view_transform, FoVPerspectiveCameras)
from pytorch3d.structures import Pointclouds
from scipy.spatial.transform import Rotation
from tqdm import tqdm

from _point_cloud import *
from _fractal import generate_point
from _point_wolf import PointWOLF

# print(isort.code(_ih[2]))

%matplotlib inline

In [ ]:
# Args
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_size: int = 224
random_seed: int = 42
point_num: int = 10000
param_path = Path("../data/point_cloud_C1000_P10000_params.pkl")

with open(param_path, "rb") as f:
    ifs_params = pickle.load(f)

print("Rendering device is", device)
print("Class num is", len(ifs_params))
print("IFS parameter is", ifs_params[0])

# Generate point cloud
point_clouds = {}
for idx in tqdm(range(len(ifs_params))):
    point_data = generate_point(ifs_params[idx], point_num)
    point_clouds[idx] = point_data.transpose()

## Point Cloud Augmentation

In [ ]:
def plot_points_3d(points_3d:np.ndarray, title:str):
    print("Point Cloud shape is", points_3d.shape)
    print("X: ", np.max(points_3d[:, 0]), np.min(points_3d[:, 0]))
    print("Y: ", np.max(points_3d[:, 1]), np.min(points_3d[:, 1]))
    print("Z: ", np.max(points_3d[:, 2]), np.min(points_3d[:, 2]))
    print(np.mean(points_3d, axis=0))

    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(projection='3d')
    ax.scatter(points_3d[:, 0], points_3d[:, 1], points_3d[:, 2], s=1)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z', labelpad=0)
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.set_zlim(-1, 1)
    ax.tick_params(labelsize = 10)
    ax.tick_params()
    plt.title(title)

In [ ]:
points_3d = point_clouds[0]
plot_points_3d(points_3d, "Original")

In [ ]:
points_3d = point_clouds[0]
augmented_point_3d = random_shift_scale_point_cloud(points_3d, prob=1.0)
plot_points_3d(augmented_point_3d, "Shift&Scale")

In [ ]:
points_3d = point_clouds[0]
augmented_point_3d = random_rotate_point_cloud(points_3d, prob=1.0)
plot_points_3d(augmented_point_3d, "Rotate")

In [ ]:
points_3d = point_clouds[0]
augmented_point_3d = random_gaussian_noise_point_cloud(points_3d, prob=1.0)
plot_points_3d(augmented_point_3d, "Gaussian Noise")

In [ ]:
augmented_point_3d = generate_point(ifs_params[0], 5 * point_num).transpose()
plot_points_3d(augmented_point_3d, "50k")

In [ ]:
points_3d = point_clouds[0]
augmented_point_3d = random_sample_point_cloud(points_3d, min_ratio=0.5, prob=1.0)
plot_points_3d(augmented_point_3d, "Random Sample")

In [ ]:
points_3d = point_clouds[0]
point_wolf = PointWOLF()
_, augmented_point_3d = point_wolf(points_3d)
plot_points_3d(augmented_point_3d, "PointWOLF")
# plot_points_3d(_, "PointWOLF")

## Rendering

In [ ]:
def render_point_cloud(points_3d: np.ndarray, image_size: int) -> np.ndarray:
    points_2d = points_3d[:, :2] * image_size / 2 + image_size / 2
    image = np.zeros((image_size, image_size), dtype=np.uint8)
    for x, y in points_2d:
        cv2.circle(
            image, (int(round(y)), int(round(x))), radius=1, color=255, thickness=-1
        )
    return image

points_3d = point_clouds[0]
render_image = render_point_cloud(points_3d, image_size)
render_image = cv2.cvtColor(render_image, cv2.COLOR_GRAY2RGB)
plt.figure(figsize=(5, 5))
plt.title("OpenCV")
plt.imshow(render_image)
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
%%timeit

render_point_cloud(points_3d, image_size)

In [ ]:
R, T = look_at_view_transform(10.0, 0.0, 0.0)
rasterizer = PointsRasterizer(
    cameras=FoVPerspectiveCameras(device=device, R=R, T=T, fov=10.0),
    raster_settings=PointsRasterizationSettings(
        radius=0.01, points_per_pixel=4
    ),
)
renderer = PointsRenderer(
    rasterizer=rasterizer,
    compositor=AlphaCompositor(
        background_color=(0., 0., 0.)
    ),
)
colors = generate_colors(points_3d)

points_3d = point_clouds[0]
_points_3d = torch.from_numpy(points_3d.astype(np.float32))
verts = torch.Tensor(_points_3d).to(device)
rgba = torch.Tensor(colors).to(device)
point_cloud = Pointclouds(points=[verts], features=[rgba])

# Render
render_image = renderer(point_cloud)[0]

# Visualize
plt.figure(figsize=(5, 5))
plt.imshow(cv2.rotate(render_image.cpu().numpy(), cv2.ROTATE_90_CLOCKWISE))
plt.title("pytorch3d")
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
%%timeit

renderer(point_cloud)

In [ ]:
R, T = look_at_view_transform(10.0, 0.0, 0.0)
rasterizer = PointsRasterizer(
    cameras=FoVPerspectiveCameras(device=device, R=R, T=T, fov=10.0),
    raster_settings=PointsRasterizationSettings(
        radius=0.02, points_per_pixel=4
    ),
)
renderer = PointsRenderer(
    rasterizer=rasterizer,
    compositor=AlphaCompositor(
        background_color=(0., 0., 0.)
    ),
)
colors = generate_colors(points_3d)

points_3d = point_clouds[0]
_points_3d = torch.from_numpy(points_3d.astype(np.float32))
verts = torch.Tensor(_points_3d).to(device)
rgba = torch.Tensor(colors).to(device)
point_cloud = Pointclouds(points=[verts], features=[rgba])

# Render
render_image = renderer(point_cloud)[0]

# Visualize
plt.figure(figsize=(5, 5))
plt.imshow(cv2.rotate(render_image.cpu().numpy(), cv2.ROTATE_90_CLOCKWISE))
plt.title("pytorch3d")
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
R, T = look_at_view_transform(10.0, 0.0, 0.0)
rasterizer = PointsRasterizer(
    cameras=FoVPerspectiveCameras(device=device, R=R, T=T, fov=10.0),
    raster_settings=PointsRasterizationSettings(
        radius=0.01, points_per_pixel=4
    ),
)
renderer = PointsRenderer(
    rasterizer=rasterizer,
    compositor=AlphaCompositor(
        background_color=(0., 0., 0.)
    ),
)
colors = colorize_point_cloud(points_3d, min_sat=0.8, min_val=0.8, random_mode=False)

points_3d = point_clouds[0]
_points_3d = torch.from_numpy(points_3d.astype(np.float32))
verts = torch.Tensor(_points_3d).to(device)
rgba = torch.Tensor(colors).to(device)
point_cloud = Pointclouds(points=[verts], features=[rgba])

# Render
render_image = renderer(point_cloud)[0]

# Visualize
plt.figure(figsize=(5, 5))
plt.imshow(cv2.rotate(render_image.cpu().numpy(), cv2.ROTATE_90_CLOCKWISE))
plt.title("pytorch3d")
plt.xticks([])
plt.yticks([])
plt.show()